El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [5]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
# Entrenamiento y evaluación de modelos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import time

In [ ]:


# Cargar el dataset
df = pd.read_csv('datasets/car_data.csv')

# Mostrar las primeras filas para verificar la carga
df.head()

# Revisión y preparación de los datos

# Revisar información general del DataFrame
print("Información general del DataFrame:")
print(df.info())
print("\nResumen estadístico de las variables numéricas:")
print(df.describe())

# Revisar valores nulos
print("\nCantidad de valores nulos por columna:")
print(df.isnull().sum())

# Eliminar columnas que no aportan valor al modelo
columnas_a_eliminar = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']
df = df.drop(columns=columnas_a_eliminar)

# Eliminar duplicados si existen
df = df.drop_duplicates()

# Corregir valores atípicos y erróneos en 'RegistrationYear'
# Consideramos años razonables entre 1950 y 2016 (según el dataset)
df = df[(df['RegistrationYear'] >= 1950) & (df['RegistrationYear'] <= 2016)]

# Corregir valores atípicos en 'Price'
# Eliminamos precios menores a 100 y mayores a 50000 euros (ajustar si es necesario)
df = df[(df['Price'] >= 100) & (df['Price'] <= 50000)]

# Corregir valores atípicos en 'Power'
# Eliminamos potencias menores a 10 CV y mayores a 500 CV (ajustar si es necesario)
df = df[(df['Power'] >= 10) & (df['Power'] <= 500)]

# Rellenar valores nulos en variables categóricas con 'unknown'
columnas_categoricas = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
for columna in columnas_categoricas:
    df[columna] = df[columna].fillna('unknown')

# Rellenar valores nulos en variables numéricas con la mediana
columnas_numericas = ['Mileage', 'RegistrationMonth']
for columna in columnas_numericas:
    df[columna] = df[columna].fillna(df[columna].median())

# Comprobar que no queden valores nulos
print("\nValores nulos después de la limpieza:")
print(df.isnull().sum())

# Codificar variables categóricas para modelos que lo requieran (por ejemplo, regresión lineal)
# Usaremos One-Hot Encoding para las variables categóricas principales
df_ohe = pd.get_dummies(df, columns=columnas_categoricas, drop_first=True)

# Separar variables objetivo y características
X = df_ohe.drop('Price', axis=1)
y = df_ohe['Price']

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Escalar las variables numéricas para modelos sensibles a la escala (por ejemplo, regresión lineal)
scaler = StandardScaler()
columnas_a_escalar = ['RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth']
X_train[columnas_a_escalar] = scaler.fit_transform(X_train[columnas_a_escalar])
X_test[columnas_a_escalar] = scaler.transform(X_test[columnas_a_escalar])

print("\nPreparación de datos completada. Listo para el entrenamiento de modelos.")



Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-nul

## **Comentarios**
 
 En esta sección, se realizó una limpieza y preparación exhaustiva de los datos, corrigiendo valores atípicos y nulos, y codificando las variables categóricas. Además, se dividió el conjunto de datos en entrenamiento y prueba, y se escalaron las variables numéricas relevantes. 

 Gracias a estos pasos, los datos están listos para ser utilizados en el entrenamiento de distintos modelos de regresión, asegurando así una mayor calidad y fiabilidad en los resultados obtenidos posteriormente.


## Entrenamiento del modelo 

In [13]:
# 1. Regresión Lineal
print("Entrenando modelo de Regresión Lineal...")
start = time.time()
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_train_time = time.time() - start

start = time.time()
y_pred_lr = lr.predict(X_test)
lr_pred_time = time.time() - start

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print(f"Regresión Lineal - RECM: {rmse_lr:.2f}, Tiempo de entrenamiento: {lr_train_time:.2f}s, Tiempo de predicción: {lr_pred_time:.4f}s\n")

# 2. Árbol de Decisión
print("Entrenando modelo Árbol de Decisión...")
start = time.time()
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
dt_train_time = time.time() - start

start = time.time()
y_pred_dt = dt.predict(X_test)
dt_pred_time = time.time() - start

rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))
print(f"Árbol de Decisión - RECM: {rmse_dt:.2f}, Tiempo de entrenamiento: {dt_train_time:.2f}s, Tiempo de predicción: {dt_pred_time:.4f}s\n")

# 3. Bosque Aleatorio
print("Entrenando modelo Bosque Aleatorio...")
start = time.time()
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_train_time = time.time() - start

start = time.time()
y_pred_rf = rf.predict(X_test)
rf_pred_time = time.time() - start

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"Bosque Aleatorio - RECM: {rmse_rf:.2f}, Tiempo de entrenamiento: {rf_train_time:.2f}s, Tiempo de predicción: {rf_pred_time:.4f}s\n")

# 4. LightGBM con ajuste de hiperparámetros
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

print("Entrenando modelo LightGBM con ajuste de hiperparámetros...")

# Definir el modelo base
lgbm = lgb.LGBMRegressor(objective='regression', random_state=42)

# Definir los hiperparámetros a probar
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [4, 6, 8]
}

# Búsqueda de hiperparámetros con validación cruzada
grid = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # Para RMSE
    cv=3,
    n_jobs=-1,
    verbose=1
)

start = time.time()
grid.fit(X_train, y_train)
lgbm_train_time = time.time() - start

# Mejor modelo encontrado
best_lgbm = grid.best_estimator_

start = time.time()
y_pred_lgbm = best_lgbm.predict(X_test)
lgbm_pred_time = time.time() - start

rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))
print(f"LightGBM - RECM: {rmse_lgbm:.2f}, Mejor combinación: {grid.best_params_}, Tiempo de entrenamiento: {lgbm_train_time:.2f}s, Tiempo de predicción: {lgbm_pred_time:.4f}s")

Entrenando modelo de Regresión Lineal...
Regresión Lineal - RECM: 2577.29, Tiempo de entrenamiento: 4.22s, Tiempo de predicción: 0.1037s

Entrenando modelo Árbol de Decisión...
Árbol de Decisión - RECM: 2060.63, Tiempo de entrenamiento: 4.48s, Tiempo de predicción: 0.0722s

Entrenando modelo Bosque Aleatorio...
Bosque Aleatorio - RECM: 1618.31, Tiempo de entrenamiento: 93.81s, Tiempo de predicción: 0.6434s

Entrenando modelo LightGBM con ajuste de hiperparámetros...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004943 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 911
[LightGBM] [Info] Number of data points in the train set: 203995, number of used features: 292
[LightGBM] [Info] Start training from score 4856.355808
[LightGBM] [Warning] No further splits with positive gain,

## Comentarios

En esta sección, se entrenaron y evaluaron varios modelos de regresión, incluyendo Regresión Lineal, Árbol de Decisión, Bosque Aleatorio y LightGBM con ajuste de hiperparámetros. Se compararon los modelos en términos de precisión (RECM) y tiempos de entrenamiento y predicción. Los resultados muestran que LightGBM y Bosque Aleatorio ofrecen un buen equilibrio entre precisión y velocidad, aunque la elección final debe considerar también la interpretabilidad y el contexto del problema. Es recomendable analizar más a fondo el modelo seleccionado y validar su desempeño en diferentes escenarios antes de implementarlo en producción.


## Análisis del modelo

In [16]:
# Análisis de los resultados de los modelos

print("Análisis de los modelos entrenados:\n")

# Comparación de los valores de RECM
print("Comparación de la calidad (RECM) de los modelos:")
print(f"- Regresión Lineal: {rmse_lr:.2f}")
print(f"- Árbol de Decisión: {rmse_dt:.2f}")
print(f"- Bosque Aleatorio: {rmse_rf:.2f}")
print(f"- LightGBM: {rmse_lgbm:.2f}")

# Identificar el mejor modelo según RECM
mejor_rmse = min(rmse_lr, rmse_dt, rmse_rf, rmse_lgbm)
if mejor_rmse == rmse_lr:
    mejor_modelo = "Regresión Lineal"
elif mejor_rmse == rmse_dt:
    mejor_modelo = "Árbol de Decisión"
elif mejor_rmse == rmse_rf:
    mejor_modelo = "Bosque Aleatorio"
else:
    mejor_modelo = "LightGBM"

print(f"\nEl modelo con mejor desempeño (menor RECM) es: {mejor_modelo}")

# Análisis de velocidad
print("\nComparación de tiempos de entrenamiento y predicción:")
print(f"- Regresión Lineal: Entrenamiento = {lr_train_time:.2f}s, Predicción = {lr_pred_time:.4f}s")
print(f"- Árbol de Decisión: Entrenamiento = {dt_train_time:.2f}s, Predicción = {dt_pred_time:.4f}s")
print(f"- Bosque Aleatorio: Entrenamiento = {rf_train_time:.2f}s, Predicción = {rf_pred_time:.4f}s")
print(f"- LightGBM: Entrenamiento = {lgbm_train_time:.2f}s, Predicción = {lgbm_pred_time:.4f}s")

print("\nConclusión:")
print(f"El modelo {mejor_modelo} ofrece el mejor equilibrio entre calidad y velocidad para este conjunto de datos. Sin embargo, es importante considerar el contexto del problema y la interpretabilidad del modelo al tomar la decisión final.")


Análisis de los modelos entrenados:

Comparación de la calidad (RECM) de los modelos:
- Regresión Lineal: 6642435.23
- Árbol de Decisión: 3485605.19
- Bosque Aleatorio: 2591204.42
- LightGBM: 1613.64

El modelo con mejor desempeño (menor RECM) es: LightGBM

Comparación de tiempos de entrenamiento y predicción:
- Regresión Lineal: Entrenamiento = 3.88s, Predicción = 0.0864s
- Árbol de Decisión: Entrenamiento = 3.33s, Predicción = 0.0530s
- Bosque Aleatorio: Entrenamiento = 91.22s, Predicción = 0.3186s
- LightGBM: Entrenamiento = 32.14s, Predicción = 0.1579s

Conclusión:
El modelo LightGBM ofrece el mejor equilibrio entre calidad y velocidad para este conjunto de datos. Sin embargo, es importante considerar el contexto del problema y la interpretabilidad del modelo al tomar la decisión final.


## **Observaciones:**  
En esta sección se compararon distintos modelos de regresión en cuanto a precisión (RECM) y tiempos de entrenamiento/predicción. Se observó que LightGBM y Bosque Aleatorio presentan un buen desempeño general, aunque la elección final debe considerar también la interpretabilidad y el contexto de uso. Es recomendable validar el modelo seleccionado en diferentes escenarios antes de su implementación definitiva.

## **Conclusión:**  
No existe un modelo universalmente superior; la mejor opción depende de las necesidades específicas del problema y de los recursos disponibles. La evaluación integral de los modelos es clave para una toma de decisiones informada.


# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos